## Dataset preprocessing
This section outlines the steps taken to preprocess the dataset before using any model. Proper preprocessing is crucial for ensuring that the data is clean, consistent, and suitable for analysis.
The dataset seems pretty clean, but we performed the following preprocessing steps to ensure data quality:


In [29]:
import pandas as pd
import os
import numpy as np

Replace "-" with 0 in the 'Gen' column to handle missing generation values.

In [30]:
def fix_gen_column(csv_path):
    df = pd.read_csv(csv_path, dtype=str)
    df.columns = df.columns.str.strip()

    if "Gen" not in df.columns:
        print(f"'Gen' column not found in {csv_path}. Skipping.")
        return

    # Replace cells that are exactly "-" (allow surrounding spaces) with "0"
    df["Gen"] = df["Gen"].replace(r"^\s*-\s*$", "0", regex=True)

    # Trim whitespace and convert to numeric, non-numeric -> NaN -> fill with 0
    df["Gen"] = pd.to_numeric(df["Gen"].str.strip(), errors="coerce").fillna(0)

    df.to_csv(csv_path, index=False)
    print("Fixed Gen column and saved:", csv_path)

**Data Type Conversion**: Convert the relevant columns to numeric types to facilitate mathematical operations and analysis. Remove $ signs and commas before conversion.

In [31]:
def convert_dollar(csv_path):
    data_hist = pd.read_csv(csv_path)

    if 'RT Busbar' not in data_hist.columns or 'RT Hub' not in data_hist.columns or \
       'DA Busbar' not in data_hist.columns or 'DA Hub' not in data_hist.columns or \
       'P/OP' not in data_hist.columns:
        print(f"One or more required columns not found in {csv_path}. Skipping.")
        return

    data_hist['RT Busbar'] = data_hist['RT Busbar'].astype(str).str.replace(r'[\(,]', '-', regex=True)
    data_hist['RT Busbar'] = data_hist['RT Busbar'].astype(str).str.replace(r'[\),]', '', regex=True)
    
    data_hist['RT Busbar'] = data_hist['RT Busbar'].astype(str).str.replace(r'[\$,]', '', regex=True)
    data_hist['RT Busbar'] = pd.to_numeric(data_hist['RT Busbar'], errors='coerce')
    
    data_hist['RT Hub'] = data_hist['RT Hub'].astype(str).str.replace(r'[\$,]', '', regex=True)
    data_hist['RT Hub'] = pd.to_numeric(data_hist['RT Hub'], errors='coerce')
    
    
    data_hist['DA Busbar'] = data_hist['DA Busbar'].astype(str).str.replace(r'[\(,]', '-', regex=True)
    data_hist['DA Busbar'] = data_hist['DA Busbar'].astype(str).str.replace(r'[\),]', '', regex=True)
    
    data_hist['DA Busbar'] = data_hist['DA Busbar'].astype(str).str.replace(r'[\$,]', '', regex=True)
    data_hist['DA Busbar'] = pd.to_numeric(data_hist['DA Busbar'], errors='coerce')
    
    data_hist['DA Hub'] = data_hist['DA Hub'].astype(str).str.replace(r'[\$,]', '', regex=True)
    data_hist['DA Hub'] = pd.to_numeric(data_hist['DA Hub'], errors='coerce')
    
    data_hist['P/OP'] = data_hist['P/OP'].astype(str).str.replace('OP' , '0', regex=True)
    data_hist['P/OP'] = data_hist['P/OP'].astype(str).str.replace('P', '1', regex=True)
    data_hist['P/OP'] = pd.to_numeric(data_hist['P/OP'], errors='coerce')

    data_hist.to_csv(csv_path, index=False)
    print("Converted dollar columns and saved:", csv_path)


**Handling Missing Values**: Check for any missing values in the dataset. If any found, just drop those rows to maintain data integrity.

In [32]:
def drop_missing_values(csv_path):  
    df = pd.read_csv(csv_path)  
    df.dropna(inplace=True)
    df.to_csv(csv_path, index=False)
    print("Dropped rows with missing values and saved:", csv_path)

Add necessary columns as features

In [33]:
def add_features(csv_path):
    df = pd.read_csv(csv_path)

    if 'Date' not in df.columns or 'HE' not in df.columns or 'Gen' not in df.columns or \
       'RT Busbar' not in df.columns or 'DA Busbar' not in df.columns or \
       'RT Hub' not in df.columns or 'DA Hub' not in df.columns:
        print(f"One or more required columns not found in {csv_path}. Skipping.")
        return -1

    # Create new features
    date = pd.to_datetime(df['Date'], format='%d-%b-%y')
    df['Month'] = date.dt.month
    df['Season'] = df['Month'].apply(lambda x: (x%12 + 3)//3)
    season_dummies = pd.get_dummies(df['Season'], prefix='Season')
    df = pd.concat([df, season_dummies], axis=1)
    df.drop(columns=['Month', 'Season'], inplace=True)  # drop intermediate columns

    df['day_of_week'] = date.dt.dayofweek
    df['day_of_year'] = date.dt.dayofyear
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

    # add vacation indicator
    # NERC holidays: New Year's Day, Memorial Day, Labor Day, Thanksgiving, and Christmas    
    df['is_vacation'] = date.apply(lambda x: 1 if 
                                   (x.month == 1 and x.day == 1) or
                                   (x.month == 5 and x.day >= 25 and x.day <= 31 and x.weekday() == 0) or
                                   (x.month == 9 and x.day >= 1 and x.day <= 7 and x.weekday() == 0) or
                                   (x.month == 11 and x.day >= 22 and x.day <= 28 and x.weekday() == 3) or
                                   (x.month == 12 and x.day >= 24 and x.day <= 31) else 0)
    
    
    # one-hot encode the HE, first split it to period of day
    df['Period_of_Day'] = pd.cut(df['HE'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)
    pod_dummies = pd.get_dummies(df['Period_of_Day'], prefix='POD')
    df = pd.concat([df, pod_dummies], axis=1)
    df.drop(columns=['Period_of_Day'], inplace=True)
    df.drop(columns=['HE'], inplace=True)  # drop HE as it's now encoded

    df.to_csv(csv_path, index=False)
    print("Added features and saved:", csv_path)


In [34]:
def remove_columns(csv_path, columns_to_remove):
    df = pd.read_csv(csv_path)
    df.drop(columns=columns_to_remove, inplace=True, errors='ignore')
    df.to_csv(csv_path, index=False)
    print("Removed specified columns and saved:", csv_path)

Preprocess all the CSV files in the data directory to ensure consistency across datasets.

In [35]:
data_dir = "../data"
# traverse all csv files in data directory and apply fixes
for filename in os.listdir(data_dir):
    if filename.endswith(".csv"):
        csv_path = os.path.join(data_dir, filename)
        fix_gen_column(csv_path)
        drop_missing_values(csv_path)
        convert_dollar(csv_path)
        add_features(csv_path)

'Gen' column not found in ../data/CAISO-Forward-Prices.csv. Skipping.
Dropped rows with missing values and saved: ../data/CAISO-Forward-Prices.csv
One or more required columns not found in ../data/CAISO-Forward-Prices.csv. Skipping.
One or more required columns not found in ../data/CAISO-Forward-Prices.csv. Skipping.
Fixed Gen column and saved: ../data/CAISO-Historical-Data.csv
Dropped rows with missing values and saved: ../data/CAISO-Historical-Data.csv
Converted dollar columns and saved: ../data/CAISO-Historical-Data.csv
Added features and saved: ../data/CAISO-Historical-Data.csv
'Gen' column not found in ../data/MISO-Forward-Prices.csv. Skipping.
Dropped rows with missing values and saved: ../data/MISO-Forward-Prices.csv
One or more required columns not found in ../data/MISO-Forward-Prices.csv. Skipping.
One or more required columns not found in ../data/MISO-Forward-Prices.csv. Skipping.
Fixed Gen column and saved: ../data/MISO-Historical-Data.csv
Dropped rows with missing values an